In [1]:
import mph
import re
import numpy as np
from scipy import integrate
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
from structure import Solve, db
from tqdm import tqdm as _tqdm

In [ ]:
from itertools import product

In [13]:
import peewee

In [2]:
client = mph.start()
model = client.load('System_batch.mph')

In [3]:
model.solve()

In [4]:
model.outer('Data')

(array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 array([0.e+00, 1.e-05, 2.e-05, 3.e-05, 4.e-05, 5.e-05, 6.e-05, 7.e-05,
        8.e-05, 9.e-05, 1.e-04]))

In [5]:
def evaluate_species(model: mph.Model, outer_number, comsol_dataset='Data'):
    reaction_node = model / 'physics' / 'Reaction Engineering'
    _reaction_node_children = [i.name() for i in reaction_node.children()]

    species = re.findall(
        string='\n'.join(_reaction_node_children),
        pattern='Species: (.*)',
    )
    species_concentration_names = ['reaction.c_' + specie for specie in species]
    row_data = model.evaluate(
        ['t'] + species_concentration_names,
        dataset=comsol_dataset,
        outer=outer_number,
    )
    return pd.DataFrame(row_data, columns=['Time'] + species)


In [6]:
df = evaluate_species(model, 3)


In [8]:
params = {
    key: float(value)
    for key,
    value in model.parameters().items()
    if 'light' not in key
}
params['light'] = 1.e-05


In [10]:
def check(string):
    if string is None:
        while string != 'q':
            string = input(f'Set {string=}, to quit - q:')
            string = string.strip()
    return string

In [22]:
def model_parametrs(model: mph.Model, changed_params: dict = {}):
    for key, value in changed_params.items():
        model.parameter(name=key, value=value)
    return model.parameters()

In [119]:
a = {
    'Ke': 1000000000.0,
    'KH': 1000000000.0,
    'Kr': 1000000000.0,
    'Kdisp': 1000000000.0,
    'KqH': 2000.0,
    'Ks': 2.0,
    'Kd': 0.05,
    'Kc': 1.0,
    'Kp': 0.001,
    'KrD': 1000000000.0,
    'Kph': 1e-05,
    'light': 1e-05,
}
a = list(a.keys())
b = [
    f"{str(i)} = DecimalField(max_digits=20, decimal_places=5, default =0)"
    for i in a
]

string = '\n'.join(b)
print(string)

Ke = DecimalField(max_digits=20, decimal_places=5, default =0)
KH = DecimalField(max_digits=20, decimal_places=5, default =0)
Kr = DecimalField(max_digits=20, decimal_places=5, default =0)
Kdisp = DecimalField(max_digits=20, decimal_places=5, default =0)
KqH = DecimalField(max_digits=20, decimal_places=5, default =0)
Ks = DecimalField(max_digits=20, decimal_places=5, default =0)
Kd = DecimalField(max_digits=20, decimal_places=5, default =0)
Kc = DecimalField(max_digits=20, decimal_places=5, default =0)
Kp = DecimalField(max_digits=20, decimal_places=5, default =0)
KrD = DecimalField(max_digits=20, decimal_places=5, default =0)
Kph = DecimalField(max_digits=20, decimal_places=5, default =0)
light = DecimalField(max_digits=20, decimal_places=5, default =0)


In [ ]:
'(SELECT group_concat(DISTINCT a) FROM my_table) as a,'

In [ ]:
SELECT 
 	GROUP_CONCAT(DISTINCT COLUMN_NAME SEPARATOR ', ')
FROM 
	information_schema.COLUMNS
WHERE `TABLE_NAME`=N'solve'

In [102]:
# TODO: logs
def solves_to_sql(
    model: mph.Model,
    name,
    desc=None,
):
    _, light_sweep = model.outer('Data')
    i = 1
    params = {
        key: float(value)
        for key,
        value in model.parameters().items()
        if 'light' not in key
    }
    params['name'] = name
    params['desc'] = desc

    notes = []
    for light_value in light_sweep:
        note = {}
        note.update(params)
        note['light'] = light_value
        df = evaluate_species(model, i)
        note['data'] = df.to_json(index=True)
        notes.append(note)

        i += 1

    with db:
        Solve.insert_many(notes).execute()


def solve_to_sql(df, params: dict, name, desc=None):
    note = params.copy()
    note['name'] = name
    note['desc'] = desc
    note['data'] = df.to_json(index=True)
    with db:
        Solve.insert(note).execute()

In [16]:
solves_to_sql(model=model, name='test_sweep')


In [19]:
solve_to_sql(df, params=params, name='test', desc='test desc')


In [25]:
model_parametrs(model)

{'Ke': '1000000000',
 'KH': '1000000000',
 'Kr': '1000000000',
 'Kdisp': '1000000000',
 'KqH': '2000',
 'Ks': '2',
 'Kd': '0.05',
 'Kc': '1',
 'Kp': '0.001',
 'KrD': '1000000000',
 'Kph': '1E-05',
 'light': '0.0001',
 'light_start': '0',
 'light_end': '1e-4',
 'light_step': '1e-5'}

In [69]:
def combinations_dict(diap: dict):
    keys = list(diap.keys())
    values = [diap[key] for key in keys]
    combinations = list(product(*values))
    result = [dict(zip(keys, comb)) for comb in combinations]
    return result

In [87]:
variative_params = combinations_dict({
    'Ks':np.linspace(0.1,5.1,11).round(5),
    'Kc':np.linspace(0.1,2.1,11).round(5),
})

In [117]:
def sweep(
    model: mph.Model,
    variable_params,
    name=None,
    desc=None,
):
    name = check(name)
    desc = check(desc)

    params_list = _tqdm(iterable=variable_params)

    tqdm_lenth =20
    i = 0
    for changed_params in params_list:
        model_parametrs(
            model=model,
            changed_params=changed_params,
        )
        model.clear()
        params_list.set_description('{:10}'.format('Solving...'))
        model.solve()
        params_list.set_description('{:10}'.format('Saving...'))
        solves_to_sql(
            model=model,
            name=name + f'_#{i}',
            desc=desc,
        )
        i+=1


In [118]:
sweep(
    model=model,
    name='sweep',
    desc='',
    variable_params=variative_params[:5],
)


Saving... : 100%|██████████| 5/5 [01:06<00:00, 13.36s/it]
